<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/R2_dentro_e_fora_da_amostra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [321]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/dados todos sem escala.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

In [322]:
# Supondo que 'dados' seja o seu DataFrame

# Converter a coluna 't' para o tipo de dado datetime, se ainda não estiver no formato adequado
dados['t'] = pd.to_datetime(dados['t'], format='%Y')

# Definir as datas de início e fim para o intervalo desejado
data_inicio = pd.to_datetime('2010')
data_fim = pd.to_datetime('2013')

# Filtrar os dados para extrair apenas as linhas no intervalo de 2010 a 2011
dados = dados.query('@data_inicio <= t <= @data_fim')

# Agora, 'dados_2010_2011' contém apenas as linhas com valores de 't' no intervalo de 2010 a 2011.


excluir setor financeiro e empresas com PL negativo, numeros faltante e transformar eventuais caracteres em números

In [323]:
# Agora, vamos remover as linhas onde 'd9' seja igual a 1
#dados = dados[dados['d11'] != 1]
# Converte a coluna 'PL2010' para números (float) e remove as linhas onde 'PL2010' seja negativo
dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')  # 'coerce' trata strings não numéricas como NaN
dados = dados[dados['PL2010'] > 0]

# Agora, vamos remover as linhas onde 'VM30Abril11' seja igual a '-'
dados = dados[dados['VM30Abril2011'] != '-']
dados = dados[dados['PL2010'] != '-']
dados = dados[dados['LL2010'] != '-']
dados = dados[dados['AT2009'] != '-']
dados = dados[dados['AT2009'] != '0']

#transforma as variáveis em números
dados['AT2009'] = pd.to_numeric(dados['AT2009'], errors='coerce')
dados['LL2010'] = pd.to_numeric(dados['LL2010'], errors='coerce')
dados['VM30Abril2011'] = pd.to_numeric(dados['VM30Abril2011'], errors='coerce')
dados['PL2010'] = pd.to_numeric(dados['PL2010'], errors='coerce')

# Lista de variáveis que você deseja escalonar
variaveis = ['VM30Abril2011', 'PL2010', 'LL2010', 'AT2009']

# Verifique o tipo de dados de cada coluna
for coluna in variaveis:
    print(f"Tipo de dados da coluna {coluna}: {dados[coluna].dtype}")


Tipo de dados da coluna VM30Abril2011: float64
Tipo de dados da coluna PL2010: float64
Tipo de dados da coluna LL2010: float64
Tipo de dados da coluna AT2009: float64


In [324]:
dados

,COD,t,VM30Abril2011,PL2010,LL2010,AT2009
0,EALT4,2010-01-01,6.072750e+04,45809.0,5927.0,143196.0
1,ALPA4,2010-01-01,3.893923e+06,1310475.0,306341.0,1748959.0
2,ABEV3,2010-01-01,1.422082e+08,24361863.0,7561383.0,40101017.0
3,AMER3,2010-01-01,2.382110e+06,225945.0,33587.0,2324320.0
4,CBEE3,2010-01-01,4.773374e+06,1583469.0,216092.0,4589050.0
...,...,...,...,...,...,...
773,WEGE3,2013-01-01,2.170456e+07,4557772.0,843467.0,8873550.0
775,WHRL4,2013-01-01,5.837070e+06,2351149.0,812747.0,5968374.0
776,PORT3,2013-01-01,2.312180e+06,1202450.0,94554.0,2556539.0
777,WLMM4,2013-01-01,5.957913e+05,479303.0,19373.0,658138.0


ESCALONAMENTO DAS VARIÁVEIS

In [325]:
#variaveis = ['VM30Abril2011', 'PL2010', 'LL2010']

#for coluna in variaveis:
 #   dados[coluna] = dados[coluna] / dados['AT2009']


WINSORIZAÇÃO: tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [326]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abril2011', 'PL2010', 'LL2010', 'AT2009']

# Converter as colunas para números, tratando não numéricos como NaN
for coluna in variaveis_winsorizar:
    dados[coluna] = pd.to_numeric(dados[coluna], errors='coerce')

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


MQO_SIMPLES

In [327]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crie o modelo de regressão linear
    modelo = sm.OLS(y_train, X_train).fit()

    # Calcule o R^2 dentro da amostra
    r2_in_sample = modelo.rsquared
    r2_in_sample_scores.append(r2_in_sample)

    # Faça previsões no conjunto de teste
    y_pred = modelo.predict(X_test)

    # Calcule o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média dos R^2 dentro e fora da amostra
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do R^2 dentro da amostra: 0.8974931065617252
Média do R^2 fora da amostra: 0.7255387518141255


MQP - MODELO SIMPLES

In [328]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo robusto usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Crie o modelo de regressão robusta
    modelo_robusto = sm.RLM(y_train, X_train, M=sm.robust.norms.HuberT()).fit()

    # Calcule o R^2 dentro da amostra
    r2_in_sample = r2_score(y_train, modelo_robusto.fittedvalues)
    r2_in_sample_scores.append(r2_in_sample)

    # Faça previsões no conjunto de teste
    y_pred = modelo_robusto.predict(X_test)

    # Calcule o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média dos R^2 dentro e fora da amostra
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do R^2 dentro da amostra: 0.894529434797588
Média do R^2 fora da amostra: 0.7406501851210263


RB_Simples (LINEAR)

In [329]:
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo Bayesian Ridge
    modelo_bayesiano = BayesianRidge()

    modelo_bayesiano.fit(X_train, y_train)
    y_pred = modelo_bayesiano.predict(X_test)

    # Calcular o erro mediano absoluto (Median Absolute Error)
    medae = median_absolute_error(y_test, y_pred)
    medae_scores.append(medae)

    # Calcular o R^2 dentro da amostra
    y_train_pred = modelo_bayesiano.predict(X_train)
    r2_in_sample = r2_score(y_train, y_train_pred)
    r2_in_sample_scores.append(r2_in_sample)

    # Calcular o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média das métricas de avaliação em todas as dobras
media_medae = np.mean(medae_scores)
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do Median Absolute Error (MedAE):", media_medae)
print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do Median Absolute Error (MedAE): 1214237.546615288
Média do R^2 dentro da amostra: 0.8974925584544928
Média do R^2 fora da amostra: 0.7259096425571964


SVR (NÃO LINEAR)

In [330]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.metrics import median_absolute_error, r2_score
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo SVR com kernel RBF
    modelo_svr = SVR(kernel='poly', degree=3, C=1.0)

    modelo_svr.fit(X_train, y_train)
    y_pred = modelo_svr.predict(X_test)

    # Calcular o erro mediano absoluto (Median Absolute Error)
    medae = median_absolute_error(y_test, y_pred)
    medae_scores.append(medae)

    # Calcular o R^2 dentro da amostra
    y_train_pred = modelo_svr.predict(X_train)
    r2_in_sample = r2_score(y_train, y_train_pred)
    r2_in_sample_scores.append(r2_in_sample)

    # Calcular o R^2 fora da amostra
    r2_out_of_sample = r2_score(y_test, y_pred)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média das métricas de avaliação em todas as dobras
media_medae = np.mean(medae_scores)
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do Median Absolute Error (MedAE):", media_medae)
print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


Média do Median Absolute Error (MedAE): 1768092.6446285523
Média do R^2 dentro da amostra: -0.03846030364732156
Média do R^2 fora da amostra: -0.06723384167508295


CART_simples (NÃO LINEAR)

In [331]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
mae_scores = []
mse_scores = []
r2_in_sample_scores = []
r2_out_of_sample_scores = []

# Treinar e avaliar o modelo usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de árvore de decisão com os parâmetros desejados
    modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)

    # Criar o modelo Bagging com 500 árvores
    bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)

    bagging_model.fit(X_train, y_train)
    y_pred = bagging_model.predict(X_test)

    # Calcular as métricas de avaliação
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2_in_sample = bagging_model.score(X_train, y_train)
    r2_out_of_sample = bagging_model.score(X_test, y_test)

    mae_scores.append(mae)
    mse_scores.append(mse)
    r2_in_sample_scores.append(r2_in_sample)
    r2_out_of_sample_scores.append(r2_out_of_sample)

# Média das métricas de avaliação em todas as dobras
media_mae = np.mean(mae_scores)
media_mse = np.mean(mse_scores)
media_r2_in_sample = np.mean(r2_in_sample_scores)
media_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print("Média do Mean Absolute Error (MAE):", media_mae)
print("Média do Mean Squared Error (MSE):", media_mse)
print("Média do R^2 dentro da amostra:", media_r2_in_sample)
print("Média do R^2 fora da amostra:", media_r2_out_of_sample)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Média do Mean Absolute Error (MAE): 2445154.07563145
Média do Mean Squared Error (MSE): 50427563785201.766
Média do R^2 dentro da amostra: 0.9855108115886466
Média do R^2 fora da amostra: 0.8446261759487428


FLORESTA ALEATÓRIA (NÃO LINEAR)

In [318]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Certifique-se de que os tipos de dados sejam numéricos
X = X.astype(float)
y = y.astype(float)

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores_insample = []
r2_scores_outsample = []

# Treinar e avaliar o modelo dentro e fora da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Floresta Aleatória com os parâmetros desejados
    random_forest_model = RandomForestRegressor(n_estimators=500, random_state=42)
    random_forest_model.fit(X_train, y_train)

    # Previsões dentro da amostra
    y_pred_insample = random_forest_model.predict(X_train)
    r2_insample = r2_score(y_train, y_pred_insample)
    r2_scores_insample.append(r2_insample)

    # Previsões fora da amostra
    y_pred_outsample = random_forest_model.predict(X_test)
    r2_outsample = r2_score(y_test, y_pred_outsample)
    r2_scores_outsample.append(r2_outsample)

# Calcular as médias das métricas
avg_medae = np.mean(medae_scores)
avg_r2_insample = np.mean(r2_scores_insample)
avg_r2_outsample = np.mean(r2_scores_outsample)

print(f'Média MAE: {avg_medae}')
print(f'Média R² dentro da amostra: {avg_r2_insample}')
print(f'Média R² fora da amostra: {avg_r2_outsample}')


Média MAE: nan
Média R² dentro da amostra: 0.988128730049743
Média R² fora da amostra: 0.8443021134106807


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


GRADIENT BOOSTING (NÃO LINEAR)

In [319]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores_insample = []
r2_scores_outsample = []

# Treinar e avaliar o modelo dentro e fora da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Regressão Gradient Boosting com os parâmetros desejados
    gradient_boosting_model = GradientBoostingRegressor(n_estimators=500, random_state=42)

    gradient_boosting_model.fit(X_train, y_train)

    # Previsões dentro da amostra
    y_pred_insample = gradient_boosting_model.predict(X_train)
    r2_insample = r2_score(y_train, y_pred_insample)
    r2_scores_insample.append(r2_insample)

    # Previsões fora da amostra
    y_pred_outsample = gradient_boosting_model.predict(X_test)
    r2_outsample = r2_score(y_test, y_pred_outsample)
    r2_scores_outsample.append(r2_outsample)

# Calcular as médias das métricas
avg_medae = np.mean(medae_scores)
avg_r2_insample = np.mean(r2_scores_insample)
avg_r2_outsample = np.mean(r2_scores_outsample)

print(f'Média MAE: {avg_medae}')
print(f'Média R² dentro da amostra: {avg_r2_insample}')
print(f'Média R² fora da amostra: {avg_r2_outsample}')


Média MAE: nan
Média R² dentro da amostra: 0.99652718324335
Média R² fora da amostra: 0.8388469654511186


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


REDES NEURAIS (NÃO LINEAR)

In [320]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL2010', 'LL2010']]
y = dados['VM30Abril2011']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []
r2_out_of_sample_scores = []  # Para armazenar R² fora da amostra

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de regressão de rede neural com os parâmetros desejados
    modelo_rede_neural = MLPRegressor(hidden_layer_sizes=(30), max_iter=1000, random_state=42)

    modelo_rede_neural.fit(X_train, y_train)
    y_pred_train = modelo_rede_neural.predict(X_train)
    y_pred_test = modelo_rede_neural.predict(X_test)

    r2_train = r2_score(y_train, y_pred_train)  # R² dentro da amostra
    r2_test = r2_score(y_test, y_pred_test)    # R² fora da amostra

    r2_scores.append(r2_train)
    r2_out_of_sample_scores.append(r2_test)

# Média dos R² dentro da amostra e fora da amostra
mean_r2_within_sample = np.mean(r2_scores)
mean_r2_out_of_sample = np.mean(r2_out_of_sample_scores)

print(f'Média R² dentro da amostra: {mean_r2_within_sample}')
print(f'Média R² fora da amostra: {mean_r2_out_of_sample}')


Média R² dentro da amostra: 0.90318979000431
Média R² fora da amostra: 0.7682028012900154
